In [125]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import scipy.stats as stats
import matplotlib.pyplot as plt
import random
import math
import csv
import datetime

#Get dataset
response = urllib.request.urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data')
html = response.read()
soup = BeautifulSoup(html,"lxml")
body=soup.find_all('p')
txt_body=str(body)
txt_body=txt_body[4:-6]
table_set=[]
for line in txt_body.splitlines():
    row=line.split(',')
    #for ndx in range (len(row)):
        #so we may check for null later...
        #if(row[ndx]=='?'):
            #row[ndx]=None
            
    table_set.append(row)
len(table_set)
len(table_set[0])
col_names=['class','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment','gill-spacing',\
            'gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring', \
             'stalk-color-above-ring', 'stalk-color-below-ring','viel-type','viel-color','ring-number','ring-type',\
                'sport-print-color','population','habitat']
           
df = pd.DataFrame(table_set, columns=col_names)
# get length of dataframe with len(df)
# get specific row with df.iloc[i] where i is row number
# get specific value from a specific row with df.iloc[i]['attribute'] where attribute is 'class', 'cap-shape', etc


In [305]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import scipy.stats as stats
import random
import math
import csv
import datetime
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score


Attbs = col_names[1:]

def splitDataset(dataf, splitRatio):
    trainSize = int(len(dataf) * splitRatio)
    testSize = int(len(dataf) * (1 - splitRatio))
    trainSet = []
    testSet = []
    df2 = pd.DataFrame(table_set, columns=col_names)
    while len(trainSet) < trainSize:
        #index = f.iloc[random.randrange(len(copy))]
        trainSet.append(dataf.iloc[random.randrange(len(dataf))])
    while len((testSet)) < testSize:
        testSet.append(dataf.iloc[random.randrange(len(dataf))])
    return [trainSet, testSet]
    

#create classifier count dictionary
def findClass(train):
    dict = {}
    for i in range(0,len(train)):
        if train.iloc[i][0] not in dict.keys():
            dict[train.iloc[i][0]] = 0
        else:
            dict[train.iloc[i][0]] += 1
    return dict

def fit(train, classifier):
    dict = {}
    Attbs = col_names[1:]
    labels = train['class'].unique
    #lab = e, p
    #for lab in labels:
    dfi = train.where(df['class'] == classifier).dropna() #classifier
    counterVar = 1;
    for attb in Attbs:
        subDict = {}
        classAndCountSeries = dfi[attb].value_counts()
        subClass = classAndCountSeries.index.values
        subCount = classAndCountSeries.values
        for i in range(0,len(subClass)):
            subDict[subClass[i]]= subCount[i]
        dict[counterVar] = subDict
        counterVar = counterVar + 1
        #print(subClass, subCount)
        #print("%s:\n" % (attb), cnts)
    return dict

def getProbability(finalDict, classD):
    Attbs = col_names[1:]
    for bigKey in finalDict.keys():
        for i in range(1, len(col_names)):
            for key in finalDict[bigKey][i].keys():
                finalDict[bigKey][i][key] /= classD[bigKey]
    return finalDict

def returnClassifier(someDict):
    trackVal = 0
    trackVar = "false"
    for key in someDict.keys():
        if someDict[key] > 0:
            returnVal = someDict[key]
            trackVar = key
    return key
            
def getResults(finalDict, testData, classDict, trainTotal):
    returnCount = 0
    totalTuples = len(testData)
    #traverse test data
    for i in range(0, len(testData)):
        #traverse attributes in testData
        subDict = {}
        for finalKey in finalDict.keys():
            tupleProbability = 1
            for j in range(1, len(col_names)):
            #getprobabilities
                subKey = testData.iloc[i][j]
                if subKey not in finalDict[finalKey][j].keys():
                    tupleProbability = 0
                    continue
                tupleProbability *= finalDict[finalKey][j][testData.iloc[i][j]]
            tupleProbability *= (classDict[finalKey] / trainTotal)
            subDict[finalKey] = tupleProbability    
        classifierIs = returnClassifier(subDict)
        if classifierIs == testData.iloc[i][0]:
            returnCount += 1
        if i == 0:
            print("    Start calculations:", datetime.datetime.now())
        if i == (len(testData) - 1):
            print("    End calculations:", datetime.datetime.now())
    print("    Total tests:", totalTuples, "    Correctly predicted:", returnCount)
    print("    Percent accuracy:", returnCount/totalTuples * 100, "percent")

def driver():
    print("Start driver: ", datetime.datetime.now())
    splitRatio = 0.67
    trainingSet, testSet = splitDataset(df, splitRatio)
    trainingSet = pd.DataFrame(trainingSet, columns=col_names)
    trainingSet.sort_index(inplace=True)
    testSet = pd.DataFrame(testSet, columns=col_names)
    testSet.sort_index(inplace=True)
    #get totals for each class and place it into dictionary
    classDict = findClass(trainingSet)
    trainTotal = 0
    for key in classDict.keys():
        trainTotal += classDict[key]
    
    #prepare nested dictionary
    colDict = {}
    for key in classDict.keys():
        colDict[key] = fit(trainingSet, key)
    #test print colDict
    #for key, values in colDict.items():
    #   print(key, values, "\n")
    #print(colDict['p']['cap-shape']['x'])
    finalDict = getProbability(colDict, classDict)
    #test print finalDict
    #for key, values in finalDict.items():
    #    print(key, values, "\n")
    #print(finalDict['p']['cap-shape']['x']) #test print nested value
    #print(testSet.iloc[1]['cap-shape'])
    print("  Training data ratio:", splitRatio)
    getResults(finalDict, testSet, classDict, trainTotal)
    
    print("  Start k-fold: ", datetime.datetime.now())
    
    #k-fold
    for i in range(0, 5):
        #Each instance is each fold
        print("  Fold",i+1)
        #Train:
        ktrainingSet, ktestSet = splitDataset(df, splitRatio/5)
        ktrainingSet = pd.DataFrame(ktrainingSet, columns=col_names)
        ktrainingSet.sort_index(inplace=True)
        kclassDict = findClass(ktrainingSet)
        ktotal = 0
        for key in kclassDict.keys():
            ktotal += kclassDict[key]
        kcolDict = {}
        for key in classDict.keys():
            kcolDict[key] = fit(ktrainingSet, key)
        kfinalDict = getProbability(kcolDict, classDict)
        #Predict:
        getResults(kfinalDict, testSet, kclassDict, ktotal)
    
    print("  End k-fold: ", datetime.datetime.now())
    
    print("End Driver: ", datetime.datetime.now())
    
driver()


Start driver:  2018-04-27 09:17:00.289112
  Training data ratio: 0.67
    Start calculations: 2018-04-27 09:17:01.983988
    End calculations: 2018-04-27 09:17:19.977992
    Total tests: 2680     Correctly predicted: 1304
    Percent accuracy is: 48.656716417910445 percent
  Start k-fold:  2018-04-27 09:17:19.977992
  Fold 1
    Start calculations: 2018-04-27 09:17:20.818257
    End calculations: 2018-04-27 09:17:38.594131
    Total tests: 2680     Correctly predicted: 1304
    Percent accuracy is: 48.656716417910445 percent
  Fold 2
    Start calculations: 2018-04-27 09:17:39.445890
    End calculations: 2018-04-27 09:17:57.471044
    Total tests: 2680     Correctly predicted: 1304
    Percent accuracy is: 48.656716417910445 percent
  Fold 3
    Start calculations: 2018-04-27 09:17:58.280756
    End calculations: 2018-04-27 09:18:16.457358
    Total tests: 2680     Correctly predicted: 1304
    Percent accuracy is: 48.656716417910445 percent
  Fold 4
    Start calculations: 2018-04-27